<a href="https://colab.research.google.com/github/Yuns-u/Boston_Housing_predict_practice/blob/main/Practice_Boston_Housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practice Deep Learning on regression problem

회귀문제를 딥러닝모델을 통해 해결해보고자 한다.
해결하고자 하는 문제는 아래와 같다.

1. Boston의 주택가격을 예측하는 딥러닝 모델을 만들어보기
2. 딥러닝이 아닌 머신러닝모델을 사용해서 같은 문제를 풀어보고 그 결과를 비교해보기

## 데이터 불러오기 및 전처리

In [ ]:
# 필요한 모듈 불러오기
import tensorflow as tf

In [ ]:
# 데이터 불러오기
boston_housing = tf.keras.datasets.boston_housing
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

In [ ]:
# 각 데이터셋의 shape 확인하기 : feature 수에 비해 데이터의 규모가 작은 것으로 보인다.
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
X_train 
#numpy.array형식이라 pandas.DataFrame처럼 상위 몇 개를 보는 방법을 사용할 수 없었다.
#하지만 pandas의 메모리 소모량을 생각했을 때 불러오지 않고 처리를 해보고 싶기도하고 pandas를 사용하지 않고 head처럼 처리할 수 있는 방법이 있을 것 같다.

In [ ]:
y_train

In [ ]:
# 데이터 정규화하기
# 데이터 정규화에 필요한 모듈 불러오기
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train) #훈련해야하는 수치들을 정규화한다.
X_test = scaler.transform(X_test) #훈련데이터가 정규화되었으므로 예측의 기반이 될 테스트데이터셋도 정규화해준다.

## 신경망을 통해 regression 문제 해결하기

In [ ]:
#모델 만들기
model_network = tf.keras.models.Sequential([
 tf.keras.layers.Dense(64, activation='relu', input_dim=13),
 tf.keras.layers.Dropout(0.2),
 tf.keras.layers.Dense(16, activation='relu'),
 tf.keras.layers.Dense(1, activation='relu')
])

In [ ]:
#모델에 컴파일러 적용하여 옵티마이저, 손실함수 등 설정해주기
model_network.compile(optimizer='adam', 
              loss='mse',
              metrics=['accuracy','mse','mae'])

In [ ]:
#학습 실행해주기
history = model_network.fit(X_train, y_train, epochs=30, batch_size=30)

In [ ]:
#모델평가하기
#R2 값으로 모델의 설명력 확인해보기
from sklearn.metrics import r2_score

y_pred = model_network.predict(X_test)

r2_score(y_test, y_pred)

## 신경망이 아닌 머신러닝모델을 사용해서 regression 문제 해결하기

In [ ]:
#선형회귀모델로 머신러닝모델 만들기

#필요한 모듈 불러오기
from sklearn.linear_model import LinearRegression #모델 만들기
from sklearn.metrics import mean_squared_error #위의 신경망과 비교하기 위한 모듈
from sklearn.metrics import mean_absolute_error #위의 신경망과 비교하기 위한 모듈

#선형회귀모델 만들기
model = LinearRegression()
#모델 학습시키기: 여러 feature들로 구성되어있으므로 다중선형회귀모델일 것이다.
model.fit(X_train, y_train)
#예측하기
y_pred = model.predict(X_test)

#예측값과 실제값 비교하여 평가하기
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE : {mae}')
print(f'MSE : {mse}')
print(f'R2 : {r2}')

## 두 모델을 비교해보기

- 신경망모델의 r2값과 다중선형회귀모델의 r2값을 비교해보면 다중선형회귀모델의 값이 더 높기 때문에 이 회귀문제에 대한 데이터의 설명력은 다중선형회귀모델이 더 좋다고 할 수 있다.

- 신경망모델의 경우 30 epoche가 될 때 쯤에서야 다중선형회귀모델과 유사한 평가지표(mse, mae)값이 나온 것을 확인할 수 있다. `loss: 21.3019 - accuracy: 0.0000e+00 - mse: 21.3019 - mae: 3.3147`에서 알 수 있다.

- 신경망 모델에서 epoche가 크다고 해서 성능이 항상 좋게 나오는 것은 아니다. 어느정도 이상에서는 epoche 횟수를 늘리는 것이 유의미한 성능 향상을 불러오는 것이 아니기 때문이다.

- 자원 리소스 측면에서 신경망 모델을 쓰는 것이 더 비효율적일 수 있다.

# 신경망에 교차검증(cross-validation) 적용해보기

## 데이터 불러오기

In [ ]:
from tensorflow.keras.datasets import boston_housing

(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

## 필요한 라이브러리들을 Import 하기

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold, StratifiedKFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## `KFold`를 통해 학습 데이터셋을 몇 개로 나눌 것인지 결정하기

In [ ]:
#두 교차검증을 모두 5개로 나누어 볼 것이다.
kf = KFold(n_splits=5)
skf = StratifiedKFold(n_splits=5, random_state=100, shuffle=True)
#훈련데이터는 feature수가 13개인 404개라는 것을 데이터 불러오기 및 전처리에서 살펴보았다.

## 모델만들기
위의 모델인 model_network를 불러와 재학습을 시켜주는 방법도 있겠지만 이미 학습이 된 상태이므로 비슷한 모델을 만들어볼 것이다.
단, 모델을 만드는 방법을 연습하기 위해 모델 함수에 층을 입력하는 방법이 아닌, 모델에 add를 사용하여 층을 넣어주는 방식으로 해보고자 한다.

In [ ]:
#필요한 모듈 불러오기
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
#모델 만들기
model_cv = Sequential()
model_cv.add(Dense(64, activation='relu'))
model_cv.add(Dense(64, activation='relu'))
model_cv.add(Dense(1)) #회귀문제로 하나의 값만 도출하면 되기 때문에 출력노드의 수를 1로 해준다.

#모델 컴파일하기
model_cv.compile(loss='mean_squared_logarithmic_error',
                 optimizer='adam',
                 metrics=['accuracy'])

In [ ]:
#모델 학습시키기
model_cv.fit(X_train, y_train, epochs=4)

## 교차검증 적용하기

In [ ]:
#k개로 나눠줄 때 데이터와 라벨의 값과 일치하도록 분리해주어야하지만 넘파이에서는 그러한 index가 없다.
#따라서 numpy.array로 되어있는 X_train,y_train을 pandas.DataFrame의 형식으로 바꿔준 뒤 학습시키는 것이 좋을 것이다.

In [ ]:
#학습데이터셋을 데이터프레임으로 바꿔주기
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

In [ ]:
#학습데이터셋을 k개의 dataset으로 나눠주기

for train_index, val_index in kf.split(np.zeros(X_train.shape[0]),y_train):
  folded_X_train = X_train.iloc[train_index,:]
  folded_X_train_label = y_train.iloc[train_index,:]
  validation_X_train = X_train.iloc[val_index,:]
  validation_X_train_label = y_train.iloc[val_index]

In [ ]:
#데이터가 잘 나뉘어졌는지 확인해보기
folded_X_train_label.head()

In [ ]:
print(folded_X_train)
print(folded_X_train.shape)

In [ ]:
#교차검증으로 모델 학습시키기
model_cv.fit(folded_X_train, folded_X_train_label,
             epochs=10,
             batch_size=64,
             validation_data=(validation_X_train, validation_X_train_label))

epoch를 더 많이 돌린다고해서 항상 손실함수값이 작아지는 것이 아니라 변동한다는 것을 위의 수치들을 통해 알 수 있다.

In [ ]:
#모델 컴파일하기
#이 때, 어떤 손실함수를 사용할 것인지 여러 가지를 실험해볼 수 있다.
#loss='binary_crossentropy'
model_cv.compile(loss='binary_crossentropy', optimizer='adam')
model_cv.fit(folded_X_train, folded_X_train_label,
             epochs=10,
             batch_size=32)

In [ ]:
#모델 컴파일하기
#이 때, 어떤 손실함수를 사용할 것인지 여러 가지를 실험해볼 수 있다.
#loss='mean_squared_error'
model_cv.compile(loss='mean_squared_error', optimizer='adam')
model_cv.fit(folded_X_train, folded_X_train_label,
             epochs=10,
             batch_size=32)

In [ ]:
#결과 확인해보기
results = model_cv.evaluate(X_test, y_test, batch_size=32)
print('mse of test loss:',results)